In [ ]:
#importar librerias
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

In [ ]:
#crear la ruta de la data
df_temp = pd.read_excel(r'./database/Temperaturas.xlsx')

In [ ]:
#analisis de la data
df_temp.head(3)

In [ ]:
df_temp.sample(2)

In [ ]:
df_temp.dtypes

In [ ]:
df_temp.tail(3)

In [ ]:
df_temp.size

In [ ]:
#crear el objeto dash
app = dash.Dash()

In [ ]:
#definición del layout de la app a partir del componente html y el core
app.layout = html.Div([
  dcc.Graph(
    id = 'graph_line'
  ),
  dcc.DatePickerRange(
    id = 'select_date',
    initial_visible_month = df_temp['fecha'].min(),
    start_date = df_temp['fecha'].min(),
    end_date = df_temp['fecha'].max()
  )  
])

In [ ]:
#funcion para los graficos y la interactividad


@app.callback(
  Output('graph_line', 'figure'),
  [
    Input('select_date', 'start_date'),
    Input('select_date', 'end_date')
  ]
)
def range_date(fmin, fmax):
  filtro_df = df_temp[(df_temp['fecha']>= fmin) & (df_temp['fecha']<= fmax)]

  #creación de trazas por ciudad
  trazas = []
  for i in filtro_df['ciudad'].unique():
    df_city = filtro_df[filtro_df['ciudad'] == i]

    trazas.append(
      go.Scatter(
        x = df_city['fecha'],
        y = df_city['temp_prom'],
        text = df_city['ciudad'],
        mode = 'lines',
        opacity = 0.7,
        marker = {'size':15},
        name = i
      )
    )
  #retornar los objetos data y layout para ser enviados al callback
  return {
    'data': trazas,
    'layout': go.Layout(
      xaxis = {'title': 'Fecha'},
      yaxis = {'title': 'Temperatura Promedio'},
      hovermode = 'closest'
    )
  }


In [ ]:
#abrir el servidor para ejecutar el codigo en del dashboard
if __name__ == "__main__":
  app.run_server(port = 7999)